## Import necessary libraries

In [1]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from tensorflow.keras.preprocessing import image

<hr>

# Step 1: Load and preprocess the dataset

**Define paths for the training and validation datasets**

In [2]:
# List directories in /content to locate the dataset
import os
import zipfile
from shutil import copy2
from sklearn.model_selection import train_test_split
os.listdir('/content')

['.config', 'sample_data']

In [3]:
!mkdir -p ~/.kaggle
!cp /content/kaggle.json  # Update with the actual path to your kaggle.json
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d salader/dogs-vs-cats

cp: missing destination file operand after '/content/kaggle.json'
Try 'cp --help' for more information.
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Dataset URL: https://www.kaggle.com/datasets/salader/dogs-vs-cats
License(s): unknown
100% 1.06G/1.06G [00:08<00:00, 206MB/s]
100% 1.06G/1.06G [00:08<00:00, 142MB/s]


In [4]:
base_dir = '/content/dogs-vs-cats'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

In [5]:
# Create directories for training and validation sets
os.makedirs(validation_dir, exist_ok=True)
os.makedirs(os.path.join(validation_dir, 'cats'), exist_ok=True)
os.makedirs(os.path.join(validation_dir, 'dogs'), exist_ok=True)

In [7]:
import zipfile
with zipfile.ZipFile('/content/dogs-vs-cats.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/dogs-vs-cats')

In [8]:
# Split dataset
for category in ['cats', 'dogs']:
    category_dir = os.path.join(train_dir, category)
    images = os.listdir(category_dir)
    train_images, val_images = train_test_split(images, test_size=0.2, random_state=42)

    # Move validation images
    for image in val_images:
        src = os.path.join(category_dir, image)
        dest = os.path.join(validation_dir, category, image)
        copy2(src, dest)

**Initialize ImageDataGenerator with augmentation for the training set**

In [9]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

**No augmentation for the validation set**

In [10]:
val_datagen = ImageDataGenerator(rescale=1./255)

In [11]:
validation_generator = val_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

Found 4000 images belonging to 2 classes.


**Create training and validation generators**

In [13]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

validation_generator = val_datagen.flow_from_directory(
    validation_dir, ##val_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

Found 20000 images belonging to 2 classes.
Found 4000 images belonging to 2 classes.


# Step 2: Load the VGG16 model without the top layers

In [14]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
base_model.trainable = False  # Freeze the convolutional base

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


# Step 3: Add custom layers for binary classification

In [15]:
x = Flatten()(base_model.output)  # Flatten the base model output
x = Dense(256, activation='relu')(x)  # Add fully connected layer
x = Dropout(0.5)(x)  # Dropout for regularization
output = Dense(1, activation='sigmoid')(x)  # Output layer for binary classification

**Define the complete model**

In [16]:
model = Model(inputs=base_model.input, outputs=output)

# Step 4: Compile the model

In [17]:
model.compile(
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.9),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Step 5: Train the model

In [18]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,  # Adjust based on resources
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


625/625 ━━━━━━━━━━━━━━━━━━━━ 170s 252ms/step - accuracy: 0.7101 - loss: 0.5449 - val_accuracy: 0.8700 - val_loss: 0.2974
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 38us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 3/10


/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


625/625 ━━━━━━━━━━━━━━━━━━━━ 191s 250ms/step - accuracy: 0.8167 - loss: 0.3962 - val_accuracy: 0.8785 - val_loss: 0.2862
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 157s 249ms/step - accuracy: 0.8241 - loss: 0.3875 - val_accuracy: 0.8910 - val_loss: 0.2649
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 19us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 203s 250ms/step - accuracy: 0.8352 - loss: 0.3612 - val_accuracy: 0.8808 - val_loss: 0.2698
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 25us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 9/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 200s 248ms/step - accuracy: 0.8424 - loss: 0.3494 - val_accuracy: 0.8873 - val_loss: 0.2493
Epoch 10/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 20us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00


# Step 6: Evaluate the model

In [19]:
val_loss, val_accuracy = model.evaluate(validation_generator)
print(f"Validation Accuracy: {val_accuracy:.2f}")

125/125 ━━━━━━━━━━━━━━━━━━━━ 9s 74ms/step - accuracy: 0.8875 - loss: 0.2508
Validation Accuracy: 0.89


# Step 7: Prediction function

In [33]:
def predict_image(img_path):
    img = image.load_img(img_path, target_size=(150, 150))  # Load image
    img_array = image.img_to_array(img) / 255.0  # Normalize image
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    prediction = model.predict(img_array)  # Predict the class
    confidence = prediction[0][0] if prediction[0][0] >= 0.5 else 1 - prediction[0][0]
    category = "Dog" if prediction[0][0] >= 0.5 else "Cat"
    return category, confidence

In [29]:
print(type('/content/Dog.jpeg'))

<class 'str'>


In [34]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image

In [35]:
from tensorflow.keras.preprocessing import image as keras_image

In [36]:
# Example usage for prediction
category, confidence = predict_image('/content/Dog.jpeg')
print(f"Predicted Category: {category} with Confidence: {confidence:.2f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Predicted Category: Dog with Confidence: 1.00
